In [1]:
!pip install torch torchvision torchaudio scikit-learn joblib tensorflow transformers --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 44.4 MB/s eta 0:00:00


In [2]:
import torch
import joblib
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import load_model
import pickle
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import os
import warnings
warnings.filterwarnings('ignore')

# Set device for PyTorch and Transformers
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')


Using device: cuda


In [3]:
# Define CNN model architecture (must match training)
import torch.nn as nn
class FCNN(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(FCNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 256)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(256, 128)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(128, num_classes)
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x

# Load vectorizer and label encoder
cnn_vectorizer = joblib.load('cnn_vectorizer.joblib')
cnn_label_encoder = joblib.load('cnn_label_encoder.joblib')

# Infer input_dim and num_classes
input_dim = cnn_vectorizer.transform(['sample']).shape[1]
num_classes = len(cnn_label_encoder.classes_)

# Load model
cnn_model = FCNN(input_dim, num_classes)
cnn_model.load_state_dict(torch.load('best_fcnn_model.pth', map_location=device))
cnn_model.to(device)
cnn_model.eval()


FileNotFoundError: [Errno 2] No such file or directory: 'cnn_vectorizer.joblib'

In [ ]:
# Load tokenizer and label encoder
with open('lstm_tokenizer.pkl', 'rb') as f:
    lstm_tokenizer = pickle.load(f)
with open('lstm_label_encoder.pkl', 'rb') as f:
    lstm_label_encoder = pickle.load(f)

# Load model
lstm_model = load_model('best_lstm_model.h5')

# Set max_len (should match training)
max_len = 40  # Change if different in your training


In [ ]:
# Load label encoder
with open('transformer_label_encoder.pkl', 'rb') as f:
    transformer_label_encoder = pickle.load(f)

# Load model and tokenizer
transformer_model = AutoModelForSequenceClassification.from_pretrained('best_transformer_model')
transformer_tokenizer = AutoTokenizer.from_pretrained('best_transformer_model')
transformer_model.to(device)
transformer_model.eval()


In [ ]:
def predict_cnn(texts):
    if isinstance(texts, str):
        texts = [texts]
    X = cnn_vectorizer.transform(texts).toarray()
    X_tensor = torch.tensor(X, dtype=torch.float32).to(device)
    with torch.no_grad():
        outputs = cnn_model(X_tensor)
        preds = torch.argmax(outputs, dim=1).cpu().numpy()
    return cnn_label_encoder.inverse_transform(preds)

def predict_lstm(texts):
    if isinstance(texts, str):
        texts = [texts]
    sequences = lstm_tokenizer.texts_to_sequences(texts)
    from tensorflow.keras.preprocessing.sequence import pad_sequences
    X = pad_sequences(sequences, maxlen=max_len, padding='post')
    preds = lstm_model.predict(X)
    pred_labels = np.argmax(preds, axis=1)
    return lstm_label_encoder.inverse_transform(pred_labels)

def predict_transformer(texts):
    if isinstance(texts, str):
        texts = [texts]
    tokens = transformer_tokenizer(texts, padding='max_length', truncation=True, max_length=64, return_tensors='pt')
    tokens = {k: v.to(device) for k, v in tokens.items()}
    with torch.no_grad():
        outputs = transformer_model(**tokens)
        preds = torch.argmax(outputs.logits, dim=1).cpu().numpy()
    return transformer_label_encoder.inverse_transform(preds)


In [ ]:
def unified_inference(texts):
    cnn_preds = predict_cnn(texts)
    lstm_preds = predict_lstm(texts)
    transformer_preds = predict_transformer(texts)
    if isinstance(texts, str):
        texts = [texts]
    print(f'{"Text":<50} | {"CNN":<10} | {"LSTM":<10} | {"Transformer":<12}')
    print('-'*90)
    for t, c, l, tr in zip(texts, cnn_preds, lstm_preds, transformer_preds):
        print(f'{t[:50]:<50} | {c:<10} | {l:<10} | {tr:<12}')


In [ ]:
sample_texts = [
    "I'm feeling so sad and empty today.",
    "I can't stop smiling, what a great day!",
    "I'm scared of what might happen next."
]
unified_inference(sample_texts)
